In [4]:
import requests
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
from pprint import pprint
import time
import warnings

In [ ]:
plt.style.use('seaborn')
warnings.filterwarnings('ignore')

# Data collection

In [ ]:
res = []

def get_vacancy(vacancies, pages=100):
    """
    The function returns a list of json-objects that contain information on
    vacancies that was found by search words from the list of vacancies.   

    Parameters:
        vacancies (list): A list of vacancies to be searched.
        pages : number of pages to be searched

    Returns:
        list of json-objects.
    """
    for vacancy in tqdm(vacancies):
        time.sleep(5)
        # print(f'collecting <{vacancy}>') 
        for page in tqdm(range(pages)):
            params = {
                     'text': f'{vacancy}',
                     'page': page,
                     'per_page': 20,
                     'only_with_salary': 'false',
                     }
            req = requests.get('https://api.hh.ru/vacancies',
                               params
                              ).json()
            # print(pd.json_normalize(req))
            if 'items' in req.keys():
                res.extend(req['items'])
                print(pd.json_normalize(req['items']))
            time.sleep(1)

vacancies = ['NAME:"machine learning"',
             'NAME:"ML engineer"',
             'NAME:"ML инженер"',
             'NAME:"ML специалист"',
             'NAME:"ML developer"',
             'NAME:"ML разработчик"',
             'NAME:"ML-специалист"',
             'NAME:"ML-разработчик"',
             'NAME:"машинное обучение"',
             'NAME:"машинному обучению"',
             'NAME:"AI developer"',
             'NAME:"AI разработчик"',
             'NAME:"MLops"',
             'NAME:"ML-ops"',
             'NAME:"data scientist"',
             'NAME:"ML ops"',
             'NAME:"NLP"',
             'NAME:"computer vision"',
             'NAME:"компьютерное зрение"',
             'NAME:"исследователь данных" NOT маркетинг',
             'NAME:"ученный по данным" NOT маркетинг',
             'NAME:"data engineer"',
             'NAME:"data analyst"',
             'NAME:"дата инженер"',
             'NAME:"дата-инженер"',
             'NAME:"аналитик данных" NOT маркетинг',
             'NAME:"аналииз данных" NOT маркетинг',
             'NAME:"big data"',
             'NAME:"больших данных"',
             'NAME:"большие данные"',
             'NAME:"нейронные сети"',
             'NAME:"нейронным сетям"',
             'NAME:"neural networks"'
            ]

get_vacancy(vacancies)

# Initial data processing

In [ ]:
data = pd.json_normalize(res)
data2 = pd.json_normalize(res, record_path=['professional_roles'])

In [ ]:
columns = ['id',
           'name',
           'professional_roles',
           'employer.name',
           'salary.from',
           'salary.to',
           'salary.currency',
           'salary.gross',
           'snippet.requirement',
           'snippet.responsibility',
           'experience.name',
           'accept_temporary',
           'area.name',
           'published_at',
           'url',
           'employment.name'
          ]

data = data[columns]

In [ ]:
data2.rename(columns={'name': 'role'}, inplace=True )
data2 = data2['role']
data = data.join(data2)

In [ ]:
# We don't need the professional_roles column any more, so we delete it.
# Колонка professional_roles нам больше не нужна, удаляем ее.  
data = data.drop(columns=['professional_roles'])

In [ ]:
# Forming a convenient table for further analysis and processing.
# Формируем удобную таблицу для дальнейшего анализа и обработки.  
columns2 = ['name',
            'role',
            'employer.name',
            'area.name',
            'salary.from',
            'salary.to',
            'salary.currency',
            'snippet.requirement',
            'snippet.responsibility',
            'experience.name',
            'accept_temporary',
            'published_at',
            'url',
            'employment.name',
            'id',
            'salary.gross'
           ]

data = data[columns2]

In [ ]:
# Let's save the data.
# Сделаем промежуточное сохранение данных.  
data.to_csv('initial_data.csv', index=False)

In [2]:
df = pd.read_csv("initial_data.csv")
df

,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,published_at,url,employment.name,id,salary.gross
0,Machine Learning Engineer,Дата-сайентист,ДОМ.РФ,Москва,NaN,NaN,NaN,Решать амбициозные задачи из области NLP. Увер...,Принимать технические и архитектурные решения ...,Нет опыта,False,2023-06-07T11:25:50+0300,https://api.hh.ru/vacancies/81645171?host=hh.ru,Полная занятость,81645171,NaN
1,Куратор на курс Start machine learning,Специалист технической поддержки,Карпов Курсес,Санкт-Петербург,50000.0,NaN,RUR,Умеете грамотно выражать свои мысли в письменн...,Модерировать чаты со студентами (до 300 челове...,От 1 года до 3 лет,False,2023-06-13T18:16:41+0300,https://api.hh.ru/vacancies/81866480?host=hh.ru,Полная занятость,81866480,False
2,Junior Data scientist/Machine learning,Аналитик,СБЕР,Москва,NaN,NaN,NaN,Обучение классических моделей машинного обучен...,Анализ и использование различных источников да...,От 1 года до 3 лет,False,2023-06-02T12:01:47+0300,https://api.hh.ru/vacancies/81456638?host=hh.ru,Полная занятость,81456638,NaN
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,250000.0,300000.0,RUR,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,True,2023-05-29T16:19:08+0300,https://api.hh.ru/vacancies/81253945?host=hh.ru,Частичная занятость,81253945,True
4,Стажер-разработчик (Machine Learning),"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Знание классических алгоритмов и структур данн...,Решение сложных задач для улучшения жизни людей.,Нет опыта,False,2023-05-26T12:10:38+0300,https://api.hh.ru/vacancies/81090082?host=hh.ru,Стажировка,81090082,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2331,"Преподаватель по курсу ""Нейронные сети""","Учитель, преподаватель, педагог",Delta LLP,Семей,150000.0,400000.0,KZT,Знания в одном из следующих направлений: Искус...,"Ведение занятий для учеников в, в небольших гр...",Нет опыта,False,2023-06-08T11:53:51+0300,https://api.hh.ru/vacancies/81700989?host=hh.ru,Полная занятость,81700989,True
2332,Разработчик в отдел обработки и анализа данных...,"Программист, разработчик",СКБ МО РФ,Санкт-Петербург,NaN,NaN,NaN,Опыт программирования на языке C++ (17/20). Оп...,Разработка новых и поддержка существующих прое...,От 3 до 6 лет,False,2023-06-16T15:30:01+0300,https://api.hh.ru/vacancies/81547576?host=hh.ru,Полная занятость,81547576,NaN
2333,Копирайтер для нейросети (в Яндекс),"Копирайтер, редактор, корректор",Гончаров Никита Дмитриевич,Омск,75000.0,NaN,RUR,Высшее образование по одной из следующих специ...,Проверять факты и отличать достоверные источни...,От 1 года до 3 лет,False,2023-06-14T11:44:52+0300,https://api.hh.ru/vacancies/81700122?host=hh.ru,Частичная занятость,81700122,True
2334,Редактор текстов для нейросети (в Яндекс),"Копирайтер, редактор, корректор",Гончаров Никита Дмитриевич,Рязань,75000.0,NaN,RUR,Высшее образование по одной из следующих специ...,Проверять факты и отличать достоверные источни...,От 1 года до 3 лет,False,2023-06-14T11:44:21+0300,https://api.hh.ru/vacancies/81700057?host=hh.ru,Частичная занятость,81700057,True


# Enrichment with new data

In [20]:
def reach_vacancy_key_skills(url):
    # Timeout set to avoid captcha.
    # Таймаут поставлен чтоб избежать капчи.  
    time.sleep(0.5) 
    req = requests.get(url).json()
    if 'key_skills' in req.keys():
        return req['key_skills']  

def reach_vacancy_languages(url):
    time.sleep(0.5)
    req = requests.get(url).json()
    if 'languages' in req.keys():
        return req['languages']

def reach_vacancy_schedule(url):
    time.sleep(0.5)
    req = requests.get(url).json()
    if 'schedule' in req.keys():
        return req['schedule']
    
def reach_vacancy_description(url):
    time.sleep(0.5)
    req = requests.get(url).json()
    print(req)
    if 'description' in req.keys():
        return req['description']    

In [ ]:
df['key_skills'] = df['url'].apply(reach_vacancy_key_skills)

In [ ]:
df['languages'] = df['url'].apply(reach_vacancy_languages)

In [13]:
df['schedule'] = df['url'].apply(reach_vacancy_schedule)

In [ ]:
df['description'] = df['url'].apply(reach_vacancy_description)

In [24]:
df.to_csv('reached_data.csv', index=False)

In [5]:
df = df = pd.read_csv("reached_data.csv")
df

,name,role,employer.name,area.name,salary.from,salary.to,salary.currency,snippet.requirement,snippet.responsibility,experience.name,accept_temporary,published_at,url,employment.name,id,salary.gross,key_skills,languages,schedule,description
0,Machine Learning Engineer,Дата-сайентист,ДОМ.РФ,Москва,NaN,NaN,NaN,Решать амбициозные задачи из области NLP. Увер...,Принимать технические и архитектурные решения ...,Нет опыта,False,2023-06-07T11:25:50+0300,https://api.hh.ru/vacancies/81645171?host=hh.ru,Полная занятость,81645171,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>ЧЕМ НЕОБХОДИМО ЗАНИМАТЬСЯ:</strong>...
1,Куратор на курс Start machine learning,Специалист технической поддержки,Карпов Курсес,Санкт-Петербург,50000.0,NaN,RUR,Умеете грамотно выражать свои мысли в письменн...,Модерировать чаты со студентами (до 300 челове...,От 1 года до 3 лет,False,2023-06-13T18:16:41+0300,https://api.hh.ru/vacancies/81866480?host=hh.ru,Полная занятость,81866480,False,"[{'name': 'Грамотность'}, {'name': 'Грамотная ...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p>Karpov.courses — молодой EdTech стартап в с...
2,Junior Data scientist/Machine learning,Аналитик,СБЕР,Москва,NaN,NaN,NaN,Обучение классических моделей машинного обучен...,Анализ и использование различных источников да...,От 1 года до 3 лет,False,2023-06-02T12:01:47+0300,https://api.hh.ru/vacancies/81456638?host=hh.ru,Полная занятость,81456638,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<p>Ищем Data scientist в Блок Сервисы. Это Бло...
3,Machine Learning Engineer (ML),Другое,Alliesverse,Москва,250000.0,300000.0,RUR,"Python, алгоритмы, математика. Знания в DL, оп...",Придумывать и внедрять новые применения для LL...,От 1 года до 3 лет,True,2023-05-29T16:19:08+0300,https://api.hh.ru/vacancies/81253945?host=hh.ru,Частичная занятость,81253945,True,"[{'name': 'ML'}, {'name': 'Алгоритмы'}]",[],"{'id': 'fullDay', 'name': 'Полный день'}",<p><strong>Задачи:</strong></p> <ul> <li>Приду...
4,Стажер-разработчик (Machine Learning),"Программист, разработчик",Яндекс,Москва,NaN,NaN,NaN,Знание классических алгоритмов и структур данн...,Решение сложных задач для улучшения жизни людей.,Нет опыта,False,2023-05-26T12:10:38+0300,https://api.hh.ru/vacancies/81090082?host=hh.ru,Стажировка,81090082,NaN,[{'name': 'Machine Learning'}],[],"{'id': 'flexible', 'name': 'Гибкий график'}",<p>Машинное обучение и нейронные сети использу...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2331,"Преподаватель по курсу ""Нейронные сети""","Учитель, преподаватель, педагог",Delta LLP,Семей,150000.0,400000.0,KZT,Знания в одном из следующих направлений: Искус...,"Ведение занятий для учеников в, в небольших гр...",Нет опыта,False,2023-06-08T11:53:51+0300,https://api.hh.ru/vacancies/81700989?host=hh.ru,Полная занятость,81700989,True,[{'name': 'JavaScript'}],[],"{'id': 'fullDay', 'name': 'Полный день'}","<p><em>Гибкий график, частичная занятость, пол..."
2332,Разработчик в отдел обработки и анализа данных...,"Программист, разработчик",СКБ МО РФ,Санкт-Петербург,NaN,NaN,NaN,Опыт программирования на языке C++ (17/20). Оп...,Разработка новых и поддержка существующих прое...,От 3 до 6 лет,False,2023-06-16T15:30:01+0300,https://api.hh.ru/vacancies/81547576?host=hh.ru,Полная занятость,81547576,NaN,[],[],"{'id': 'fullDay', 'name': 'Полный день'}",<strong>Обязанности:</strong> <ul> <li>Разрабо...
2333,Копирайтер для нейросети (в Яндекс),"Копирайтер, редактор, корректор",Гончаров Никита Дмитриевич,Омск,75000.0,NaN,RUR,Высшее образование по одной из следующих специ...,Проверять факты и отличать достоверные источни...,От 1 года до 3 лет,False,2023-06-14T11:44:52+0300,https://api.hh.ru/vacancies/81700122?host=hh.ru,Частичная занятость,81700122,True,"[{'name': 'Редактирование текстов'}, {'name': ...",[],"{'id': 'remote', 'name': 'Удаленная работа'}",<p><strong>Работа тренером/редактором искусств...
2334,Редактор текстов для нейросети (в Яндекс),"Копирайтер, редактор, корректор",Гон